In [1]:
import plotly
plotly.tools.set_credentials_file(username='JeroenEijkens', api_key='GII1TUJzIKwxofWRmvWa')

Let $w : [0,T] \to [1,\infty)$ be a non-decreasing $C^1$-function with the required integrability conditions. We are working on the space $$H_w^T = \{ h \in L^1_{loc}([0,T]) : \|h\|_w^2 := |h(0)|^2 + \int_0^T |h'(x)|^2w(x) dx < \infty\}.$$

In [11]:
import numpy as np
import math
from scipy.integrate import quad
from scipy.integrate import trapz
from scipy.integrate import cumtrapz
from itertools import product
import plotly.plotly as py
import plotly.graph_objs as go

# Possible weight functions, where alpha has to be positive
# and beta greater than 3.
alpha = 0.1
beta = 3

def w1(x):
    return np.exp(alpha*x)

def w2(x):
    return (1+x)**beta

# Nelson-Siegel initial curve
z1 = 7.69
z2 = -4.13
z3 = -2.2
z4 = 3
def r0(x):
    return z1 + (z2 + z3*x)*np.exp(-z4*x)

# Kernel on L^2
a = 0.01
def kernel_B(x,y):
    return np.exp(-a*(x-y))

# Numerical derivative of a function f
def d_fun(f):
    h = 1e-15
    return (lambda x: (f(x+h)-f(x))/h)

## Diffusion B
$H_w^T$ is isometrically isomorphic to $R \times L^2[0,T]$, by the isomorphism $$T(h) = (h(0), h'w^{1/2}), \quad T^{-1}(c,f)(u) = c + \int_0^u f(y) w^{-1/2}(y) dy.$$
A Hilbert-Schmidt operator $\hat{B}$ on $L^2$ is a Hilbert-Schmidt integral operator with respect to some square-integrable kernel $k(x,y)$. A Hilbert-Schmidt operator $\tilde{B}$ on $R \times L^2$ can be defined by
$$\tilde{B}(c,f) = (bc, \hat{B}f).$$
Therefore we can represent a Hilbert-Schmidt operator $B$ on $H_w$ by 
$$Bh = T^{-1} \tilde{B} Th = bh(0) + \int_0^\cdot \int_0^\infty k(u,y) h'(y) w^{1/2}(y) dy w^{-1/2}(u) du.$$
Since $T$ is an isometry, $T^* = T^{-1}$. If $\hat{B}$ is self-adjoint, that is when $k(x,y) = k(y,x)$, then $B^* = B$.
We also require that $B$ is $H_w^0$-valued. The image of $H_w^0$ under $T$ is the orthogonal complement to $(1,w^{-1/2})$ in $R \times L^2$. Define
$$S : R \times L^2 \to R \times L^2 : (c,f) \mapsto (c,f)-\frac{c+\langle w^{-1/2}, f\rangle_{L^2}}{1+\langle w^{-1/2},w^{-1/2}\rangle_{L^2}}(1,w^{-1/2}).$$
We will compute $B =T^{-1} S \tilde{B} T$, which is now a $H_w^0$-valued Hilbert-Schmidt operator on $H_w$. As $S$ is an orthogonal projection, it is self-adjoint. Hence $B^* = T^{-1} \tilde{B} ST$ if $\tilde{B}$ is self-adjoint.

In [12]:
b = 0
w = w1
k = kernel_B
def w1_min(x):
    return np.exp(-alpha*x)
def w1_min_half(x):
    return np.exp(-0.5*alpha*x)

def B_HS(h,T_bound,dx,weight,kernel_B,b):
    """
    Compute Bh(x) for all values x in the interval [0,T_bound], with steps of size dx.
    
    Args: 
        h (function): 
        T_bound (float): Upper limit of integration of interval [0,T]
        dx (float): Stepsize of L2[0,T] integration
        weight (function): 
        kernel_B (function of 2 variables): 
        b (float): 
        
    Comments:
        By default the weight function is w(x) = exp(-alpha*x). For other weight functions
        you should change the definition of w1_min and w1_min_half above as expected. 
        These definitions are required, as exp(alpha*x) grows very quickly and one cannot
        use w1**(-1) instead.
    """
    x_space = np.arange(0,round(T_bound+dx,15),dx)
    xx, yy = np.meshgrid(x_space,x_space)
    integrand = kernel_B(yy,xx)*d_fun(h)(xx)*w(xx)**(1/2)
    integrated = trapz(integrand, x_space)
    int_fw = cumtrapz(integrated*w1_min_half(x_space),x_space,initial=0)
    int_ww = cumtrapz(w1_min(x_space),x_space,initial=0)
    return b*h(0)-(b*h(0) + int_fw[-1])/(1 + int_ww[-1]) + \
            int_fw - (b*h(0) + int_fw[-1])/(1+int_ww[-1])*int_ww
    
def B_HS_star(h,T_bound,dx,weight,kernel_B,b):
    """
    Compute B*h(x) for all values x in the interval [0,T_bound], with steps of size dx.
    See B_HS for further explanations.
    """
    x_space = np.arange(0,round(T_bound+dx,15),dx)
    int_ww = trapz(w1_min(x_space),x_space)
    constant = h(T_bound)/(1+int_ww)
    xx, yy = np.meshgrid(x_space,x_space)
    integrand = kernel_B(yy,xx)*(d_fun(h)(xx)*w(xx)**(1/2) - constant*w1_min_half(xx))
    integral = trapz(integrand,x_space)
    return b*(h(0) + constant) + cumtrapz(integral*w1_min_half(x_space),x_space,initial=0)

def T_inv(c,f,T_bound,dx,weight):
    """ Computes T^(-1)(c,f)(x) evaluated at all points x in [0,T_bound] with stepsize dx.
    
    Args:
        c (float):
        f (function): 
        T_bound (float):
        dx (float): 
        weight (function): 
    """
    x_space = np.arange(0,round(T_bound+dx,15),dx)
    T_inv = c + cumtrapz(f(x_space)*w1_min_half(x_space),x_space,initial=0)
    return T_inv


HJM Equation: $$df_t = (Af_t + a)dt + BdW(t).$$

Augmenting $X(t) = (f_t, r(t))$, which has dynamics
$$dX(t) = (A' X(t) + a') dt + B'dW(t),$$
where $a' = (a,a(0))$, $B'u = (Bu, \delta_0 Bu)$, $A'(x,x_\omega) = (x', x'(0))$. The adjoint semigroup $S'^*$ is given by
$$S'^*(t)(x,x_\omega) = (S^*(t)x + \hat{x}_{\omega,t}, x_\omega),$$
where
$$S^*(t)x(u) = x(0) + x(0)\int_0^{u\land t} \frac{1}{w(s)}ds + \int_t^{u \lor t} \frac{x'(s-t)w(s-t)}{w(s)} ds,$$
and
$$\hat{x}_{\omega}(t,u) = x_\omega \int_0^{u \land t} \frac{1}{w(s)}ds.$$
The adjoint of $B'$ is given by $B'^*(x,x_\omega) = B^*(x+x_omega)$.
We want to first solve

$$\partial_t \Psi(t,u) = A'^*\Psi(t,u) - (0,1), \quad \Psi(0,u) = (u,0)$$
So we compute
$$\Psi(t,u) = S'^*(t) (u,0) - \int_0^t S'^*(t-s)(0,1)ds = (S^*(t)u - \int_0^t \hat{1}_s ds,t).$$

These values are only sensical when t << T.

In [62]:
def one_hat(T_bound,dx,weight):
    """
    Compute int_0^(min(t,x)) 1/w(y) dy for all values in the grid [0,T_bound] x [0,T_bound] with
    stepsize dx.
    
    Args:
        T_bound (float):
        dx (float):
        weight (function):
    """
    # In order to avoid round-off errors we add 1e-15
    n = int((T_bound+dx+1e-15)/dx)
    one_hat = np.zeros((n,n))
    integrand = w1_min(np.arange(0,round(T_bound+dx,15),dx))
    for i in range(1,n):
        one_hat[i:,i:] = one_hat[i:,i:] + 0.5*(integrand[i-1]+integrand[i])*dx
    return one_hat

def one_hat_integrated(T_bound,dx,weight):
    """
    Compute integral_0^t one_hat(t-s)(x)ds for all values (t,x) in the grid [0,T_bound] x [0,T_bound] with 
    stepsize dx.
    """
    n = int((T_bound+dx+1e-15)/dx)
    one_hat_integrated = np.zeros((n,n))
    one_hat_values = one_hat(T_bound,dx,weight)
    for i in range(0,n):
        one_hat_integrated[:,i] = cumtrapz(one_hat_values[:,i],np.arange(0,round(T_bound+dx,15),dx),initial=0)
    return one_hat_integrated

def S_star(h,T_bound,dx,weight):
    """
    Compute S*(t)h(x) for all values (t,x) in the grid [0,T_bound] x [0,T_bound] with stepsize dx.
    
    Args: 
        h (function):
        T_bound (float):
        dx (float):
        weight (function):  
    """
    n = int((T_bound+dx+1e-15)/dx)
    x_space = np.arange(0,round(T_bound+dx,15),dx)
    integrand = np.zeros((n,n),dtype=complex)
    for i in range(0,n):
        for j in range(0,n):
            integrand[i,j] = d_fun(h)(x_space[j])*w(x_space[j])/(w(x_space[j]+x_space[i]))
    S_star = np.zeros((n,n))
    for i in range(0,n-1):
        S_star[i,i:] = cumtrapz(integrand[i,0:(n-i)],x_space[0:(n-i)],initial=0)
    S_star = h(0) + h(0)*one_hat(T_bound,dx,weight)+S_star
    return S_star

def Psi(h,T_bound,dx,weight):
    """
    Compute Psi1(t,h)(x) = S'^*(t)h(x) - int_0^t hat(1)_s(x) ds for all values (t,x) in the grid [0,T_bound] x [0,T_bound] with
    stepsize dx. 
    """
    return (S_star(h,T_bound,dx,weight) - one_hat_integrated(T_bound,dx,weight),\
            -np.arange(0,round(T_bound+dx,15),dx))

Let $\{e_n\}$ be an orthonormal basis of $\mathbb{R}\times L^2([0,T])$, then $\{T^{-1}(e_n)\}$ is an orthonormal basis of $H_w$. In order to satisfy the HJM drift condition, we require that
$$a = \sum_{j} \mathcal{S} B(e_j),$$
where $\mathcal{S}h(x) = h(x) \int_0^x h(s) ds.$ 

In [39]:
def basis(n,T_bound,x):
    """
    Compute nth orthonormal basis vector of R x L2[0,T_bound] in x, for n=0,1,2,...
    
    Args: 
        n (natural number): 
        T_bound (float):
        x (float):
    """
    if n%2 == 0:
        if n == 0:
            return 0
        else:
            return np.sqrt(2/T_bound)*np.sin(n*np.pi*x/(2*T_bound))
    else:
        if n == 1:
            return np.sqrt(1/T_bound)
        else:
            return np.sqrt(2/T_bound)*np.cos((n-1)*np.pi*x/(2*T_bound))

def basis_w(n,T_bound,dx,weight):
    """
    Compute nth orthonormal basis vector of H_w for all values in the interval [0,T_bound]
    with stepsize dx, for n=0,1,2,...
    
    Args: 
        n (natural number):
        T_bound (float):
        dx (float):
        weight (function):
    """
    if n==0:
        return T_inv(1,lambda x: basis(n,T_bound,x),T_bound,dx,weight)
    else:
        return T_inv(0,lambda x: basis(n,T_bound,x),T_bound,dx,weight)

def B_basis(n,T_bound,dx,weight,kernel_B,b):
    """
    Compute Be_n for nth orthonormal basis vector of H_w for all values in the interval [0,T_bound]
    with stepsize dx, for n=0,1,2,...
    
    Args:
        n (natural number):
        T_bound (float):
        dx (float):
        weight (function):
        kernel_B (function of 2 variables):
        b (float):
    """
    basis_w_fun = lambda x: np.interp(x, np.arange(0,round(T_bound+dx,5),dx),basis_w(n,T_bound,dx,weight))
    return B_HS(basis_w_fun,T_bound,dx,weight,kernel_B,b)

def HJM_drift(T_bound,dx,weight,kernel_B,b,N):
    """
    Compute HJM_drift function for all values in the interval [0,T_bound] with stepsize dx
    as determined by the Hilbert-Schmidt operator B.
    
    Args: 
        N (natural number): number of basis functions
        T_bound (float): 
        dx (float):
        weight (function):
        kernel_B (function of 2 variables):
        b (float):
    """
    HJM_drift = 0
    x_space = np.arange(0,round(T_bound+dx,15),dx)
    for n in range(0,N):
        HJM_drift = HJM_drift + B_basis(n,T_bound,dx,weight,kernel_B,b)* \
                        cumtrapz(B_basis(n,T_bound,dx,weight,kernel_B,b),x_space,initial=0)
    return HJM_drift

Next we compute
$$ \partial_t \Phi(t,u) = \tfrac{1}{2} \langle B'QB'^* \Psi(t,u), \overline{\Psi(t,u)}\rangle_{H'_{\mathbb{C}}} + \langle \Psi(t,u),a'\rangle_{H'_{\mathbb{C}}}.$$

In [47]:
def BQB_Psi(h,T_bound,dx,weight,kernel_B,b):
    """
    Compute B'QB'* Psi(t,h)(x) for all values (t,x) in the grid [0,T_bound] x [0,T_bound] with
    stepsize dx.
    """
    psi = Psi(h,T_bound+3*dx,dx,weight)
    psi1 = psi[0]
    psi2 = psi[1]
    x_space = np.arange(0,round(T_bound+dx,15),dx)
    n = np.size(x_space)
    BQB_Psi = np.zeros((n,n))
    for i in range(0,n):
        psi_fun = lambda x: np.interp(x,np.arange(0,round(T_bound+4*dx,15),dx),psi1[i,:] + psi2[i])
        B_psi = B_HS(psi_fun,T_bound+2*dx, dx,weight,kernel_B,b)
        B_psi_fun = lambda x: np.interp(x,np.arange(0,round(T_bound+3*dx,15),dx),B_psi)
        QB_psi = B_HS(B_psi_fun,T_bound+dx,dx,weight,kernel_B,b)
        QB_psi_fun = lambda x: np.interp(x,np.arange(0,round(T_bound+2*dx,15),dx),QB_psi)
        BQB_Psi[i,:] = B_HS(QB_psi_fun,T_bound,dx,weight,kernel_B,b)
    return (BQB_Psi, BQB_Psi[:,0])

def ip_BQB_psi_psi(h,T_bound,dx,weight,kernel_B,b):
    """
    Compute 1/2 <B'QB'* Psi(t,h),Psi(t,h)> for all t in the interval [0,T_bound] with stepsize dx.
    """
    x_space = np.arange(0,round(T_bound+dx,15),dx)
    n = np.size(x_space)
    psi = Psi(h,T_bound+dx,dx,weight)
    psi1 = psi[0]
    psi2 = psi[1]
    BQB_psi = BQB_Psi(h,T_bound+dx,dx,weight,kernel_B,b)
    BQB_psi1 = BQB_psi[0]
    BQB_psi2 = BQB_psi[1]
    ip_BQB_psi_psi = np.zeros(n)
    for i in range(0,n):
        psi1_fun = lambda x: np.interp(x,np.arange(0,round(T_bound+2*dx,15),dx),psi1[i,:])
        d_psi1 = d_fun(psi1_fun)(x_space)
        BQB_psi1_fun = lambda x: np.interp(x,np.arange(0,round(T_bound+2*dx,15),dx),BQB_psi1[i,:])
        d_BQB_psi1 = d_fun(BQB_psi1_fun)(x_space)
        ip_BQB_psi_psi[i] = 0.5*(BQB_psi2[i]*psi2[i] + BQB_psi1[i,0]*psi1[i,0] + \
                                 trapz(d_psi1*d_BQB_psi1*weight(x_space),x_space))
    return ip_BQB_psi_psi

def ip_psi_drift(h,T_bound,dx,weight,kernel_B,b,N):
    """
    Compute <Psi(t,h),a'> for all t in the interval [0,T_bound] with stepsize dx.
    """
    x_space = np.arange(0,round(T_bound+dx,15),dx)
    n = np.size(x_space)
    a = HJM_drift(T_bound+dx,dx,weight,kernel_B,b,N)
    a_fun = lambda x: np.interp(x,np.arange(0,round(T_bound+2*dx,15),dx),a)
    d_a = d_fun(a_fun)(x_space)
    psi = Psi(h,T_bound+dx,dx,weight)
    psi1 = psi[0]
    psi2 = psi[1]
    ip_psi_drift = np.zeros(n)
    for i in range(0,n):
        psi1_fun = lambda x: np.interp(x,np.arange(0,round(T_bound+2*dx,15),dx),psi1[i,:])
        d_psi1 = d_fun(psi1_fun)(x_space)
        ip_psi_drift[i] = psi2[i]*a[0] + psi1[i,0]*a[0] + trapz(d_psi1*d_a*weight(x_space),x_space)
    return ip_psi_drift

def Phi(h,T_bound,dx,weight,kernel_B,b,N):
    """
    Compute Phi(t,h) for all values t in the interval [0,T_bound] with stepsize dx.
    """
    t_space = np.arange(0,round(T_bound+dx,15),dx)
    d_phi = ip_BQB_psi_psi(h,T_bound,dx,weight,kernel_B,b) + ip_psi_drift(h,T_bound,dx,weight,kernel_B,b,N)
    return cumtrapz(d_phi,t_space,initial=0)

Now we can turn to computing e.g. the price of a call option on a forward rate $(f(t,T)-K)^+$. Let $K>0$. For any $y \in R$ the following identities hold:
$$\frac{1}{2\pi} \int_{R} e^{(w+iz)y} \frac{K^{-(w-1+iz)}}{(w+iz)(w-1+iz)} d z = \begin{cases}
(K-e^y)^+ & \text{if $w<0$}, \\
(e^y - K)^+ - e^y & \text{if $0<w<1$}, \\
(e^y - K)^+ &\text{if $w>1$}.
\end{cases}$$

$$\pi(0) = \int_R e^{\Phi(t,(w+iz)\delta_{T-t}) + \langle \Psi(t,(w+iz)\delta_{T-t}),f(0,\cdot)\rangle_{H'}} \tilde{F}(z) dz $$

The dual element of $\delta_{T-t}$ in $H_w$ is $x \mapsto x(1+\int_0^{(T-t)\land x} \frac{1}{w(u)}du)$.

In [64]:
def f_tilde(w,K,z):
    return (K**(-(w-1+1j*z)))/((w+1j*z)*(w-1+1j*z))

def delta(T_bound,x,dx,weight):
    """
    Compute the dual function of delta_x(y) for all values y in the interval [0,T_bound] with stepsize dx.
    """
    x_space = np.arange(0,round(T_bound+dx,15),dx)
    index_x = int((x+1e-15)/dx)
    integral = one_hat(T_bound,dx,weight)[index_x,:]
    return x_space*(1+integral)

def price(t,x,f0,T_bound,dx,weight,kernel_B,b,N,R,w,K):
    delta_x = delta(T_bound,x,dx,weight)
    x_space = np.arange(0,round(T_bound+dx,15),dx)
    delta_x_fun = lambda y: np.interp(y,x_space,delta_x)
    index_t = int((t+1e-15)/dx)
    d_f0 = d_fun(f0)(x_space)
    z_space = np.arange(-R,R,dx)
    n = np.size(z_space)
    integrand = np.zeros(n)
    for i in range(0,n):
        z = z_space[i]
        delta_x_fun2 = lambda y: (w+1j*z)*delta_x_fun(y)
        phi = Phi(delta_x_fun2,T_bound,dx,weight,kernel_B,b,N)[index_t]
        psi = Psi(delta_x_fun2,T_bound,dx,weight)
        psi1 = psi[0][index_t,:]
        psi2 = psi[1][index_t]
        psi1_fun = lambda y: np.interp(y, x_space,psi1)
        d_psi1 = d_fun(psi1_fun)(x_space)
        ip_psi_f0 = psi2*f0(0) + psi1[0]*f0(0) + trapz(d_psi1*d_f0*weight(x_space),x_space)
        integrand[i] = np.exp(phi + ip_psi_f0)*f_tilde(w,K,z_space[i])
    return trapz(integrand,z_space)

price(5,2,r0,10,0.1,w1,kernel_B,b,5,10,2,2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:49: ComplexWarning:

Casting complex values to real discards the imaginary part

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: ComplexWarning:

Casting complex values to real discards the imaginary part

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:39: ComplexWarning:

Casting complex values to real discards the imaginary part

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:58: ComplexWarning:

Casting complex values to real discards the imaginary part

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:32: RuntimeWarning:

overflow encountered in exp

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:32: ComplexWarning:

Casting complex values to real discards the imaginary part

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4351: RuntimeWarning:

invalid value encountered in add



nan

In [45]:
# Create a plot
t = 10
dx = 0.1
x_space = np.arange(0,round(t+dx,15),dx)
y_space = Phi(r0,10,0.1,w1,kernel_B,b,10)
y2_space = S_star(r0,10,0.1,w1)[5,:]
    
trace1 = go.Scatter(
    x = x_space,
    y = y_space,
    mode = 'lines',
    name = 'Increments BM',
)
trace2 = go.Scatter(
    x = x_space,
    y = y2_space,
    mode = 'lines',
    name = 'Orthogonal Expansion BM'
)

data = [trace1]

# Edit the layout
layout = dict(title = 'Two sample paths of Brownian motion using cumulative summation of increments',
             xaxis = dict(title = 'Time'),
             yaxis = dict(title = 'B(t)'),
             )

fig = dict(data=data, layout=layout)

py.iplot(fig, filename='Brownian Motions')